# Financial Data APIs: A Step-by-Step Guide to Financial Data Collection and Integration

### Introduction

#### What Are Financial Data APIs?

Financial data APIs (Application Programming Interfaces) are tools that allow software applications to access and use financial information from different sources. Imagine they are like bridges that let different systems communicate and share data about things like stock prices, currency exchange rates, company financials, or market trends. These APIs help businesses, developers, and even individuals get up-to-date financial data without needing to manually look up or collect it themselves.

#### Why Do We Use Financial Data APIs?

1. **Instant Access to Data**: Instead of manually searching for financial information or waiting for reports to be published, financial data APIs provide real-time access to up-to-date data at the click of a button. This makes it easy to get the information you need quickly, without delays.

2. **Automation and Efficiency**: Financial data APIs enable you to automate the process of retrieving and updating financial data. For example, a business can automatically pull the latest stock prices or currency exchange rates without needing a person to check them every day. This saves time and reduces human error.

3. **Accuracy and Reliability**: By connecting to trusted, authoritative data sources, APIs ensure that the financial information you're using is accurate and reliable.

4. **Seamless Integration**: Financial data APIs allow different software systems to communicate with each other and integrate easily. For example, a financial management tool can use APIs to fetch market data, track investments, or manage transactions from a variety of sources—all in one place.


### Tutorial overview

In this tutorial, you will be interacting with five different financial data APIs which are either free to use or have a free version. These are:

- OpenFIGI
- PermID
- GLEIF
- AlphaVantage
- FinancialModelingPrep

The main data problem that we will be resolving in this tutorial concerns the collection and aggregation of financial identifier data (aka reference data), market data, and fundamental data for a handful of financial stocks.

In finance, identifying a financial instrument such as a stock, bond, or derivative may appear simple. However, a single asset can be associated with multiple identifiers, depending on the system, region, or intended use. See the image below to have an idea of the vast financial identifier landscape

<img src="https://github.com/TamerKhraisha/FinancialDataEngineering/blob/main/conferences/DSC2024/images/financial_identifiers_landscape.png?raw=true" alt="image info" width="600">

Moreover, these identifiers vary in their level of detail, scope, accessibility, availability, and popularity.

Additionally, information such as the company name, industry sector, financial instrument category, as well as fundamental and market data like prices and revenue figures, may be required for comprehensive financial identification.

This creates a challenges: data using different financial identifiers cannot be easily matched. 

Fortunately, this issue is well-recognized, and financial markets have been working to address it. Various initiatives have been introduced, such as the ISO establishing a dedicated committee to oversee the creation of reference data standards. Bloomberg launched OpenFigi, and LSEG (formerly Refinitiv) introduced PermID, essentially open-sourcing their proprietary identifiers.

Let's get started!

### Before you start: make sure you have the required API keys

Consult the README file of this project to see how to get your free API keys for AlphaVantage, FinancialModelingPrep, and PermID. One you have the keys, assign them to the variables below

In [117]:
alphavantage_api_key = "" 
financialmodelingprep_api_key = ""
permid_api_key = ""

### Package installation
As usual, let's start with importing the necessary packages for this tutorial

In [118]:
import pandas as pd
import json
import os
import urllib
import pprint
import requests
import io

### Initial data

Our initial data consists of the ticker symbols for the following five companies:
- Apple Inc. (AAPL)
- Microsoft Corporation (MSFT)
- Alphabet Inc. (GOOGL)
- Amazon.com, Inc. (AMZN)
- NVIDIA Corporation (NVDA)

In [119]:
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA"]

### Get FIGI identifiers

A FIGI (Financial Instrument Global Identifier) is a unique identifier assigned to financial instruments like stocks, bonds, derivatives, loans, funds, and others. It is used globally to identify instruments across markets and data systems. FIGIs are available at three levels:

- **Global Share Class Level**: Identifies a financial instrument at the share class level, meaning that all shares of the same class, regardless of where or how they are traded, share the same FIGI.

- **Country Composite Level**: Identifies a financial instrument as it is traded within a specific country, regardless of which exchange or venue within that country it is traded on.

- **Exchange/Venue Level**: Identifies the specific financial instrument as traded on a particular exchange or trading venue.


![image info](https://www.openfigi.com/assets/images/figi-tree-1fbb8484e1.png)

For details on openFIGI API, see the official [webpage](https://www.openfigi.com/api)

In [120]:
mic_code = 'XNYS' # We want identifiers for the New York Stock Exchange market
id_type = 'TICKER' # We are submitting tickers for which we want the FIGI identifiers

For details on exchange codes and their meanings, see the downlaodable file called Exchange Codes in this [page](https://www.openfigi.com/about/figi).

In [121]:
jobs = [ {'idType': id_type, 'idValue': ticker, 'micCode': mic_code} for ticker in tickers ]

In [122]:
pprint.pprint(jobs)

[{'idType': 'TICKER', 'idValue': 'AAPL', 'micCode': 'XNYS'},
 {'idType': 'TICKER', 'idValue': 'MSFT', 'micCode': 'XNYS'}]


Having defined the jobs, we can now write a function to submit requests to the OpenFIGI API to get the FIGI identifiers. This function includes an optional parameter for an API key. The API key is not mandatory for using FIGI, but possessing one will provide you with higher rate limits.

In [123]:
def map_jobs(jobs: list[dict], openfigi_apikey:str=None):
    handler = urllib.request.HTTPHandler()
    opener = urllib.request.build_opener(handler)
    openfigi_url = 'https://api.openfigi.com/v2/mapping'
    request = urllib.request.Request(openfigi_url, data=bytes(json.dumps(jobs), encoding='utf-8'))
    request.add_header('Content-Type','application/json')
    if openfigi_apikey:
        request.add_header('X-OPENFIGI-APIKEY', openfigi_apikey)
    request.get_method = lambda: 'POST'
    connection = opener.open(request)
    if connection.code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return json.loads(connection.read().decode('utf-8'))

Now let's submit our jobs and explore the results

In [124]:
figi_job_results = map_jobs(jobs)
pprint.pprint(figi_job_results)

[{'data': [{'compositeFIGI': 'BBG000B9XRY4',
            'exchCode': 'UN',
            'figi': 'BBG000B9XVV8',
            'marketSector': 'Equity',
            'name': 'APPLE INC',
            'securityDescription': 'AAPL',
            'securityType': 'Common Stock',
            'securityType2': 'Common Stock',
            'shareClassFIGI': 'BBG001S5N8V8',
            'ticker': 'AAPL',
            'uniqueID': None,
            'uniqueIDFutOpt': None}]},
 {'data': [{'compositeFIGI': 'BBG000BPH459',
            'exchCode': 'UN',
            'figi': 'BBG000BPH654',
            'marketSector': 'Equity',
            'name': 'MICROSOFT CORP',
            'securityDescription': 'MSFT',
            'securityType': 'Common Stock',
            'securityType2': 'Common Stock',
            'shareClassFIGI': 'BBG001S5TD05',
            'ticker': 'MSFT',
            'uniqueID': None,
            'uniqueIDFutOpt': None}]}]


The mapping job yields a list of dictionaries, each with a parent key named "data". To simplify converting this to a Pandas DataFrame, we’ll adjust the structure to exclude the 'data' keys and keep only their corresponding values.

In [125]:
figi_job_result_dictionaries = [d['data'][0] for d in figi_job_results]

With this new list of dictionaries, we can directly convert it to a Pandas dataframe

In [126]:
df_figi = pd.DataFrame.from_dict(figi_job_result_dictionaries)
df_figi

,figi,name,ticker,exchCode,compositeFIGI,uniqueID,securityType,marketSector,shareClassFIGI,uniqueIDFutOpt,securityType2,securityDescription
0,BBG000B9XVV8,APPLE INC,AAPL,UN,BBG000B9XRY4,None,Common Stock,Equity,BBG001S5N8V8,None,Common Stock,AAPL
1,BBG000BPH654,MICROSOFT CORP,MSFT,UN,BBG000BPH459,None,Common Stock,Equity,BBG001S5TD05,None,Common Stock,MSFT


We don’t need all these fields, so let’s retain only the ones we want, discard the rest, and rename them using lowercase with underscores.

In [127]:
df_figi.rename(columns={"exchCode": "exchange_code", 
                        "compositeFIGI": "composite_figi",
                        "securityType": "security_type",
                        "marketSector": "market_sector",
                        "shareClassFIGI": "share_class_figi",
                        "securityType": "security_type"},
              inplace=True)
df_figi.drop(columns=["uniqueID", "uniqueIDFutOpt", "securityDescription", "securityType2"], inplace=True)
df_figi = df_figi[["name", "ticker", "exchange_code", "security_type", "figi", "market_sector", "composite_figi", "share_class_figi"]]
df_figi

,name,ticker,exchange_code,security_type,figi,market_sector,composite_figi,share_class_figi
0,APPLE INC,AAPL,UN,Common Stock,BBG000B9XVV8,Equity,BBG000B9XRY4,BBG001S5N8V8
1,MICROSOFT CORP,MSFT,UN,Common Stock,BBG000BPH654,Equity,BBG000BPH459,BBG001S5TD05


🎉 Great! We’ve successfully mapped tickers to their corresponding FIGIs and obtained additional information about our company stocks. Now, let’s proceed to the next step: PermIDs.

### Get PermID identifiers

[Permanent Identifier](https://developers.lseg.com/en/api-catalog/open-perm-id/permid-record-matching-restful-api/documentation/overview-and-concepts/faq#:~:text=Permanent%20Identifier%20(PermID)%20uniquely%20identifies,that%20often%20change%20over%20time.) (PermID) uniquely identifies objects in the LSEG Information Model. The model includes organizations, instruments, funds, issuers and people and defines how these objects are linked together – linkages that often change over time. Information available through the PermID system is segmented into multiple levels, including:



- **Organization Level**: identifies organizations such as companies, institutions, or legal entities.

- **Quote Level**: can be used to identify specific stock quotes or pricing information for an instrument on different exchanges.

- **Instrument Level**: identifies individual financial instruments like stocks, bonds, derivatives, and funds.


![image info](https://github.com/TamerKhraisha/FinancialDataEngineering/blob/main/conferences/ODSC2024/images/permid.png?raw=true)

For details on openFIGI API, see the official [webpage](https://www.openfigi.com/api)

In [128]:
# This is a small test to make sure that PermID API key has been properly set as an environmental variable
if "PERMID_API_KEY" not in os.environ:
    print("Missing PermID key!")
else:
    print("Your PermID key has been correctly set")

Your PermID key has been correctly set


In [129]:
def get_permid_data_using_search_api(tickers: list[str], mic: str):
    results = {}
    for ticker in tickers:
        url = f'https://api-eit.refinitiv.com/permid/search?q=ticker:{ticker}%20AND%20mic:{mic}'
        access_token = os.environ['PERMID_API_KEY']
        headers = {'X-AG-Access-Token' : access_token}
        try:
            response = requests.get(url, headers=headers)
        except Exception  as e:
            print ('Error in connect ' , e)
            return
        if response.status_code == 200:
            results[ticker] = response.json()
    return results

In [130]:
permid_search_api_results = get_permid_data_using_search_api(tickers=tickers, mic=mic_code)

In [131]:
permid_search_api_results["AAPL"]

{'result': {'organizations': {'entityType': 'organizations',
   'total': 1,
   'start': 1,
   'num': 1,
   'entities': [{'@id': 'https://permid.org/1-4295905573',
     'organizationName': 'Apple Inc',
     'primaryTicker': 'AAPL',
     'orgSubtype': 'Company',
     'hasHoldingClassification': 'publiclyHeld',
     'hasURL': 'https://www.apple.com/'}]},
  'instruments': {'entityType': 'instruments',
   'total': 1,
   'start': 1,
   'num': 1,
   'entities': [{'@id': 'https://permid.org/1-8590932301',
     'hasName': 'Apple Ord Shs',
     'assetClass': 'Ordinary Shares',
     'isIssuedByName': 'Apple Inc',
     'isIssuedBy': 'https://permid.org/1-4295905573',
     'hasPrimaryQuote': 'https://permid.org/1-55835312773',
     'primaryTicker': 'AAPL'}]},
  'quotes': {'entityType': 'quotes',
   'total': 1,
   'start': 1,
   'num': 1,
   'entities': [{'@id': 'https://permid.org/1-25727408109',
     'hasName': 'APPLE ORD',
     'assetClass': 'Ordinary Shares',
     'isQuoteOfInstrumentName': 'App

Let's parse the data and keep the fields we are interested in

In [132]:
dict_permid_data = {}
for ticker in tickers:
    dict_permid_data[ticker] = {"ticker": ticker}
    dict_permid_data[ticker]["org_permid_url"] = permid_search_api_results[ticker]["result"]["organizations"]["entities"][0]["@id"]
    dict_permid_data[ticker]["org_permid_id"] = permid_search_api_results[ticker]["result"]["organizations"]["entities"][0]["@id"].split("-")[1]
    dict_permid_data[ticker]["instrument_permid_url"] = permid_search_api_results[ticker]["result"]["instruments"]["entities"][0]["@id"]
    dict_permid_data[ticker]["instrument_permid_id"] = permid_search_api_results[ticker]["result"]["instruments"]["entities"][0]["@id"].split("-")[1]
    dict_permid_data[ticker]["quote_permid_url"] = permid_search_api_results[ticker]["result"]["quotes"]["entities"][0]["@id"]
    dict_permid_data[ticker]["quote_permid_id"] = permid_search_api_results[ticker]["result"]["quotes"]["entities"][0]["@id"].split("-")[1]
    dict_permid_data[ticker]["ric"] = permid_search_api_results[ticker]["result"]["quotes"]["entities"][0]["hasRIC"]
    dict_permid_data[ticker]["asset_class"] = permid_search_api_results[ticker]["result"]["instruments"]["entities"][0]["assetClass"]
    dict_permid_data[ticker]["org_url"] = permid_search_api_results[ticker]["result"]["organizations"]["entities"][0]["hasURL"]
    dict_permid_data[ticker]["holding_classification"] = permid_search_api_results[ticker]["result"]["organizations"]["entities"][0]["hasHoldingClassification"]

In [133]:
pprint.pprint(dict_permid_data)

{'AAPL': {'asset_class': 'Ordinary Shares',
          'holding_classification': 'publiclyHeld',
          'instrument_permid_id': '8590932301',
          'instrument_permid_url': 'https://permid.org/1-8590932301',
          'org_permid_id': '4295905573',
          'org_permid_url': 'https://permid.org/1-4295905573',
          'org_url': 'https://www.apple.com/',
          'quote_permid_id': '25727408109',
          'quote_permid_url': 'https://permid.org/1-25727408109',
          'ric': 'AAPL.N',
          'ticker': 'AAPL'},
 'MSFT': {'asset_class': 'Ordinary Shares',
          'holding_classification': 'publiclyHeld',
          'instrument_permid_id': '8590921450',
          'instrument_permid_url': 'https://permid.org/1-8590921450',
          'org_permid_id': '4295907168',
          'org_permid_url': 'https://permid.org/1-4295907168',
          'org_url': 'https://www.microsoft.com/en-us',
          'quote_permid_id': '25727407231',
          'quote_permid_url': 'https://permid.org/1

In [134]:
def permid_additional_info(permid_url: str):
    permid_headers = {
        'Accept': 'text/turtle',
    }

    permid_params = {
        'format': 'json-ld',
        'access-token': os.environ['PERMID_API_KEY']
    }

    # The actual request
    permid_response = requests.get(permid_url, headers=permid_headers, params=permid_params)
    
    # Convert the response to JSON
    permid_data = json.loads(permid_response.content)
    
    return permid_data

In [135]:
for ticker in tickers:
    additional_info = permid_additional_info(permid_url=dict_permid_data[ticker]["org_permid_url"])
    dict_permid_data[ticker]['ipo'] = additional_info['hasIPODate']
    dict_permid_data[ticker]['address'] = additional_info['mdaas:HeadquartersAddress']
    dict_permid_data[ticker]['phone'] = additional_info['tr-org:hasHeadquartersPhoneNumber']
    dict_permid_data[ticker]['lei'] = additional_info['tr-org:hasLEI']

In [136]:
pprint.pprint(dict_permid_data)

{'AAPL': {'address': 'One Apple Park Way\n'
                     'CUPERTINO\n'
                     'CALIFORNIA\n'
                     '95014\n'
                     'United States\n',
          'asset_class': 'Ordinary Shares',
          'holding_classification': 'publiclyHeld',
          'instrument_permid_id': '8590932301',
          'instrument_permid_url': 'https://permid.org/1-8590932301',
          'ipo': '1980-12-12T05:00:00Z',
          'lei': 'HWUPKR0MPOU8FGXBT394',
          'org_permid_id': '4295905573',
          'org_permid_url': 'https://permid.org/1-4295905573',
          'org_url': 'https://www.apple.com/',
          'phone': '14089961010',
          'quote_permid_id': '25727408109',
          'quote_permid_url': 'https://permid.org/1-25727408109',
          'ric': 'AAPL.N',
          'ticker': 'AAPL'},
 'MSFT': {'address': 'One Microsoft Way\n'
                     'REDMOND\n'
                     'WASHINGTON\n'
                     '98052-6399\n'
                   

## Get more info from the GLEIF API

**GLEIF (Global Legal Entity Identifier Foundation)** is an organization established to support the implementation and use of the Legal Entity Identifier (LEI) system.

[GLEIF API](https://www.gleif.org/en/lei-data/gleif-api) gives access to LEI identifiers and related corporate data

In this tutorial, we will be retreiving the BIC codes for our sample of five companies using their LEI identifiers which we got from PermID

In [137]:
def query_lei_records(lei: str, page_size: int=10, page_number:int=1):
    # Define the endpoint URL
    url = f"https://api.gleif.org/api/v1/lei-records?page[size]={page_size}&page[number]={page_number}&filter[lei]={lei}"    
    try:
        # Send a GET request to the endpoint
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            
            # Extract the "data" field from the response
            lei_records = data.get("data")
            
            # Return the lei_records data
            return lei_records
        else:
            print(f"Failed to fetch LEI records. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [138]:
apple_lei_data = query_lei_records(lei=dict_permid_data["AAPL"]["lei"])
pprint.pprint(apple_lei_data)

[{'attributes': {'bic': ['APLEUS66XXX'],
                 'conformityFlag': 'CONFORMING',
                 'entity': {'associatedEntity': {'lei': None, 'name': None},
                            'category': 'GENERAL',
                            'creationDate': '1977-01-03T00:00:00Z',
                            'eventGroups': [],
                            'expiration': {'date': None, 'reason': None},
                            'headquartersAddress': {'addressLines': ['ONE '
                                                                     'APPLE '
                                                                     'PARK '
                                                                     'WAY'],
                                                    'addressNumber': None,
                                                    'addressNumberWithinBuilding': None,
                                                    'city': 'CUPERTINO',
                                                

In [139]:
for ticker in tickers:
    dict_permid_data[ticker]["bic"] = query_lei_records(lei=dict_permid_data[ticker]["lei"])[0].get("attributes").get("bic")

### Prepare the final df with permids and gleif data

In [140]:
df_permid_gleif = pd.DataFrame(dict_permid_data.values())

### Merge FIGI data

In [175]:
final_reference_dataframe = df_figi.merge(df_permid_gleif, on="ticker")
final_reference_dataframe

,name,ticker,exchange_code,security_type,figi,market_sector,composite_figi,share_class_figi,org_permid_url,org_permid_id,...,quote_permid_id,ric,asset_class,org_url,holding_classification,ipo,address,phone,lei,bic
0,APPLE INC,AAPL,UN,Common Stock,BBG000B9XVV8,Equity,BBG000B9XRY4,BBG001S5N8V8,https://permid.org/1-4295905573,4295905573,...,25727408109,AAPL.N,Ordinary Shares,https://www.apple.com/,publiclyHeld,1980-12-12T05:00:00Z,One Apple Park Way\nCUPERTINO\nCALIFORNIA\n950...,14089961010,HWUPKR0MPOU8FGXBT394,[APLEUS66XXX]
1,MICROSOFT CORP,MSFT,UN,Common Stock,BBG000BPH654,Equity,BBG000BPH459,BBG001S5TD05,https://permid.org/1-4295907168,4295907168,...,25727407231,MSFT.N,Ordinary Shares,https://www.microsoft.com/en-us,publiclyHeld,1986-03-13T05:00:00Z,One Microsoft Way\nREDMOND\nWASHINGTON\n98052-...,14258828080,INR2EJN1ERAN0W5ZP974,[MSFTUS66XXX]


### Get market data from AlphaVantage

[AlphaVantage](https://www.alphavantage.co/) is a financial data provider offering real-time and historical APIs for stock market data, options, forex, and crypto, along with over 60 technical and economic indicators, market news and sentiment analysis, and extensive global coverage across asset classes.


In [142]:
def get_market_data_by_ticker(ticker):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize=compact&datatype=csv&adjusted=true&apikey={alphavantage_api_key}'
    request_result = requests.get(url)
    df = pd.read_csv(io.StringIO(request_result.text), sep=',')
    df['ticker'] = ticker
    return df

In [156]:
alphavantage_data_df = pd.DataFrame()
for ticker in tickers:
    ticker_data = get_market_data_by_ticker(ticker)
    alphavantage_data_df = pd.concat([alphavantage_data_df, ticker_data], axis=0)
alphavantage_data_df.reset_index(inplace=True, drop=True)

In [187]:
alphavantage_data_df.rename(columns={"timestamp": "market_data_date"}, inplace=True)

#### Let's see a sample of the data


In [188]:
alphavantage_data_df.head()

,market_data_date,open,high,low,close,volume,ticker
0,2024-11-15,226.40,226.92,224.270,225.00,47923696,AAPL
1,2024-11-14,225.02,228.87,225.000,228.22,44923941,AAPL
2,2024-11-13,224.01,226.65,222.760,225.12,48566217,AAPL
3,2024-11-12,224.55,225.59,223.355,224.23,40398299,AAPL
4,2024-11-11,225.00,225.70,221.500,224.23,42005602,AAPL


#### Let's filter the data to take the most recent price

In [189]:
# Get the index of the max date for each ticker
max_date_idx = alphavantage_data_df.groupby('ticker')['market_data_date'].idxmax()

# Filter the DataFrame using these indices
filtered_alphavantage_data_df = alphavantage_data_df.loc[max_date_idx]

In [190]:
filtered_alphavantage_data_df

,market_data_date,open,high,low,close,volume,ticker
0,2024-11-15,226.40,226.92,224.27,225.0,47923696,AAPL
100,2024-11-15,419.82,422.80,413.64,415.0,28247644,MSFT


## Get fundamentals data from FinancialModelingPrep

[FinancialModelingPrep](https://site.financialmodelingprep.com/) offers APIs to access a wide range of financial data, including:

- Stock Market Data: Realtime and historical data for global stocks, ETFs, and mutual funds.
- Fundamental Data: Access to company profiles, financial statements (income statement, balance sheet, cash flow), and key financial metrics.
- Market Indices & Currencies: Data for popular indices and foreign exchange rates.
- Cryptocurrency Data: Realtime and historical data for various cryptocurrencies.
- Technical Indicators: APIs for various technical indicators (e.g., SMA, EMA, MACD).
- Earnings & Economic Data: Access to earnings calendars, economic indicators, and market news.

In [161]:
def get_fundamentals_data_by_ticker(ticker):
    url = f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period=annual&apikey={financialmodelingprep_api_key}'
    response = requests.get(url)
    response_data = response.json()
    return response_data

#### Fetch the data

In [162]:
fmp_data_dict = {}
for ticker in tickers:
    fmp_data_dict[ticker] = get_fundamentals_data_by_ticker(ticker)

#### See a sample of the data

In [164]:
fmp_data_dict["AAPL"][0]

{'date': '2024-09-28',
 'symbol': 'AAPL',
 'reportedCurrency': 'USD',
 'cik': '0000320193',
 'fillingDate': '2024-11-01',
 'acceptedDate': '2024-11-01 06:01:36',
 'calendarYear': '2024',
 'period': 'FY',
 'revenue': 391035000000,
 'costOfRevenue': 210352000000,
 'grossProfit': 180683000000,
 'grossProfitRatio': 0.4620634982,
 'researchAndDevelopmentExpenses': 31370000000,
 'generalAndAdministrativeExpenses': 0,
 'sellingAndMarketingExpenses': 0,
 'sellingGeneralAndAdministrativeExpenses': 26097000000,
 'otherExpenses': 0,
 'operatingExpenses': 57467000000,
 'costAndExpenses': 267819000000,
 'interestIncome': 0,
 'interestExpense': 0,
 'depreciationAndAmortization': 11445000000,
 'ebitda': 134661000000,
 'ebitdaratio': 0.3443707085,
 'operatingIncome': 123216000000,
 'operatingIncomeRatio': 0.3151022287,
 'totalOtherIncomeExpensesNet': 269000000,
 'incomeBeforeTax': 123485000000,
 'incomeBeforeTaxRatio': 0.3157901467,
 'incomeTaxExpense': 29749000000,
 'netIncome': 93736000000,
 'netInc

#### We convert the raw data into a Pandas dataframe

In [192]:
fmp_df = pd.DataFrame()
for ticker in tickers:
    ticker_df = pd.DataFrame([fmp_data_dict[ticker][0]])
    fmp_df = pd.concat([fmp_df, ticker_df], axis=0)
fmp_df.rename(columns={"symbol": "ticker", "fillingDate": "fundamental_data_filling_date", "netIncome": "net_income", "reportedCurrency": "reporting_currency"}, inplace=True)

#### Let's take a few items to build our final dataset

In [193]:
fmp_df_filtered = fmp_df[["ticker", "cik", "revenue", "reporting_currency", "net_income", "fundamental_data_filling_date"]]

In [194]:
fmp_df_filtered

,ticker,cik,revenue,reporting_currency,net_income,fundamental_data_filling_date
0,AAPL,0000320193,391035000000,USD,93736000000,2024-11-01
0,MSFT,0000789019,245122000000,USD,88136000000,2024-07-30


#### Let's merge our market and fundamentals data

In [195]:
market_and_fundamentals_df = fmp_df_filtered.merge(filtered_alphavantage_data_df, on="ticker")

In [196]:
market_and_fundamentals_df

,ticker,cik,revenue,reporting_currency,net_income,fundamental_data_filling_date,market_data_date,open,high,low,close,volume
0,AAPL,0000320193,391035000000,USD,93736000000,2024-11-01,2024-11-15,226.40,226.92,224.27,225.0,47923696
1,MSFT,0000789019,245122000000,USD,88136000000,2024-07-30,2024-11-15,419.82,422.80,413.64,415.0,28247644


#### Now let's merge everything into one dataframe

In [200]:
reference_fundamental_market_data = final_reference_dataframe.merge(market_and_fundamentals_df, on="ticker")

#### Let's see the final outcome

In [203]:
reference_fundamental_market_data.query("ticker == 'AAPL'").to_dict(orient='records')[0]

{'name': 'APPLE INC',
 'ticker': 'AAPL',
 'exchange_code': 'UN',
 'security_type': 'Common Stock',
 'figi': 'BBG000B9XVV8',
 'market_sector': 'Equity',
 'composite_figi': 'BBG000B9XRY4',
 'share_class_figi': 'BBG001S5N8V8',
 'org_permid_url': 'https://permid.org/1-4295905573',
 'org_permid_id': '4295905573',
 'instrument_permid_url': 'https://permid.org/1-8590932301',
 'instrument_permid_id': '8590932301',
 'quote_permid_url': 'https://permid.org/1-25727408109',
 'quote_permid_id': '25727408109',
 'ric': 'AAPL.N',
 'asset_class': 'Ordinary Shares',
 'org_url': 'https://www.apple.com/',
 'holding_classification': 'publiclyHeld',
 'ipo': '1980-12-12T05:00:00Z',
 'address': 'One Apple Park Way\nCUPERTINO\nCALIFORNIA\n95014\nUnited States\n',
 'phone': '14089961010',
 'lei': 'HWUPKR0MPOU8FGXBT394',
 'bic': ['APLEUS66XXX'],
 'cik': '0000320193',
 'revenue': 391035000000,
 'reporting_currency': 'USD',
 'net_income': 93736000000,
 'fundamental_data_filling_date': '2024-11-01',
 'market_data_d

## That's it for this tutorial

<img src="https://github.com/TamerKhraisha/FinancialDataEngineering/blob/main/conferences/DSC2024/images/party.png?raw=true" alt="image info" width="200">

##### We started with just one field—company tickers—and a handful of ticker symbols, but now we’ve expanded our data to include 23 fields that include company name, FIGI, PermID, Legal Entity Identifier (LEI), Business Identifier Code (BIC), IPO date, physical address, website, phone number, and more.

### Not bad, right?